In [6]:
import os
import pandas as pd
import datetime
from datetime import date
import cx_Oracle
import configuration as config

In [7]:
'''Conexión a la base de datos'''
#cx_Oracle.init_oracle_client(lib_dir=config.lib_dir)
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding=config.encoding)
    # show the version of the Oracle Database
    print(connection.version)
except cx_Oracle.Error as error:
    print("error:", error)

12.1.0.2.0


### Consultas personas HOY

In [8]:
# numero de asignaciones facturadas x persona
time_start = datetime.datetime.now()

cur = connection.cursor()
cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_FACTURADAS END AS ASIGNACIONES_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_FACTURADAS \
FROM ASIGNACIONES A \
INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
WHERE AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
AND   AH.FECHA_FIN < CAST(CURRENT_TIMESTAMP AS DATE) \
AND   A.TARIFA_HORA > 0 \
GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NULL \
AND   P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
AND  P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_facturas_persona = pd.DataFrame(res , columns=['PERSONA', 'ASIGNACIONES_FACTURADAS'])
df_facturas_persona.head(2)

Duración de Consulta (seg): 0:00:00.677621


,PERSONA,ASIGNACIONES_FACTURADAS
0,6,0
1,8,0


In [9]:
# numero de asignaciones no facturadas x persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_NO_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_NO_FACTURADAS END AS ASIGNACIONES_NO_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_NO_FACTURADAS \
FROM ASIGNACIONES A \
INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
WHERE AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
AND   AH.FECHA_FIN < CAST(CURRENT_TIMESTAMP AS DATE) \
AND   A.TARIFA_HORA = 0 \
GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NULL \
AND   P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
AND   P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_nofacturas_persona = pd.DataFrame(res , columns=['PERSONA', 'ASIGNACIONES_NO_FACTURADAS'])
df_nofacturas_persona.head(2)

Duración de Consulta (seg): 0:00:00.640302


,PERSONA,ASIGNACIONES_NO_FACTURADAS
0,6,14
1,8,17


In [10]:
#Asignaciones de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_TIPOS_SERVICIO IS NULL THEN 0 ELSE B.CANTIDAD_TIPOS_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND P.FECHA_RETIRO IS NULL \
    AND P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
    AND  P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(S.TIPO) AS CANTIDAD_TIPOS_SERVICIO \
	FROM ASIGNACIONES A \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO  = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON A.CON_PERSONA  = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON A.COD_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-12) \
	AND   AH.FECHA_FIN < CAST(CURRENT_TIMESTAMP AS DATE) \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiposervicio_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_TIPOS_SERVICIO'])
pv_tiposervicio_persona = pd.pivot_table(df_tiposervicio_persona, values='CANTIDAD_TIPOS_SERVICIO', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_tiposervicio_persona.columns = [str(col) + '_TP' for col in pv_tiposervicio_persona.columns]
pv_tiposervicio_persona.head(2)

Duración de Consulta (seg): 0:00:07.778646


,AC_TP,ADM_TP,AI_TP,AP_TP,CO_TP,CU_TP,GC_TP,HD_TP,IN_TP,LI_TP,NE_TP,OP_TP,PD_TP,PM_TP,PR_TP,RE_TP,SE_TP,SM_TP,SO_TP,TD_TP
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,13,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,1,14,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0


In [11]:
#Horas de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.HORAS_TIPO_SERVICIO IS NULL THEN 0 ELSE B.HORAS_TIPO_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND P.FECHA_RETIRO IS NULL \
    AND P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
    AND  P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	S.TIPO                 AS TIPO_SERVICIO, \
	SUM(EA.DURACION_HORAS) AS HORAS_TIPO_SERVICIO \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO        = S.CODIGO_SERVICIO \
	INNER JOIN PERSONAS  P             ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE P.ESTADO = 1 \
	AND EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
	AND   EA.HORA_FINALIZACION < CAST(CURRENT_TIMESTAMP AS DATE) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_tiposervicio_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','HORAS_TIPO_SERVICIO'])
pv_horas_tiposervicio_persona = pd.pivot_table(df_horas_tiposervicio_persona, values='HORAS_TIPO_SERVICIO', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_horas_tiposervicio_persona.columns = [str(col) + '_HR' for col in pv_horas_tiposervicio_persona.columns]
pv_horas_tiposervicio_persona.head(2)

Duración de Consulta (seg): 0:00:12.046504


,AC_HR,ADM_HR,AI_HR,AP_HR,CO_HR,CU_HR,GC_HR,HD_HR,IN_HR,LI_HR,NE_HR,OP_HR,PD_HR,PM_HR,PR_HR,RE_HR,SE_HR,SM_HR,SO_HR,TD_HR
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0.0,292.00,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
8,0.0,522.83,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0


In [12]:
#Tecnologias usada en asignaciones por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.HERRAMIENTA, CASE WHEN B.VECES_HERRAMIENTA_ASIGNADA IS NULL THEN 0 ELSE B.VECES_HERRAMIENTA_ASIGNADA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA,H.NOMBRE AS HERRAMIENTA \
	FROM HERRAMIENTAS H	 \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND   P.FECHA_RETIRO IS NULL \
    AND   P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
    AND  P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
	GROUP BY P.CONSECUTIVO,H.NOMBRE) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO  AS PERSONA,H.NOMBRE  AS HERRAMIENTA,COUNT(AH.CONSECUTIVO) AS VECES_HERRAMIENTA_ASIGNADA \
	FROM HERRAMIENTAS_UTILIZADAS HU \
	INNER JOIN HERRAMIENTAS H             ON HU.CON_HERRAMIENTAS = H.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON HU.COD_SERVICIO     = S.CODIGO_SERVICIO \
	INNER JOIN ASIGNACIONES A             ON S.CODIGO_SERVICIO   = A.COD_SERVICIO \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO       = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON P.CONSECUTIVO       = A.CON_PERSONA \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
	AND   AH.FECHA_FIN < CAST(CURRENT_TIMESTAMP AS DATE) \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, H.NOMBRE) B ON A.PERSONA = B.PERSONA AND  A.HERRAMIENTA = B.HERRAMIENTA \
ORDER BY A.PERSONA, A.HERRAMIENTA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tecno_asig_persona = pd.DataFrame(res , columns=['PERSONA', 'HERRAMIENTA','VECES_HERRAMIENTA_ASIGNADA'])
pv_tecno_asig_persona = pd.pivot_table(df_tecno_asig_persona, values='VECES_HERRAMIENTA_ASIGNADA', index=['PERSONA'], columns='HERRAMIENTA', fill_value=0)
pv_tecno_asig_persona.head(2)

Duración de Consulta (seg): 0:01:06.341634


HERRAMIENTA,.NET,.NET Core,ANGULAR,AWS,Activemq,Alibaba,Android / Java,Android / Kotlin,Angular,Apache Airflow,...,Travis CI,TypeScript,Vue,WEBLOGIC,Websphere,Websphere Message Broker,Wordpress,Xamarin,iOS / Objetive C,iOS / Swift
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#Horas por etapa por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.ETAPA, CASE WHEN B.HORAS_ETAPA IS NULL THEN 0 ELSE B.HORAS_ETAPA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, E.CODIGO_ETAPA  AS ETAPA \
	FROM ETAPAS  E \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
    AND   P.FECHA_RETIRO IS NULL \
    AND   P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
    AND  P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
	GROUP BY P.CONSECUTIVO, E.CODIGO_ETAPA) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	EA.CODIGO_ETAPA        AS ETAPA, \
	SUM(EA.DURACION_HORAS) AS HORAS_ETAPA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN PERSONAS P ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE P.ESTADO = 1 \
	AND   EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
	AND   EA.HORA_FINALIZACION < CAST(CURRENT_TIMESTAMP AS DATE) \
	GROUP BY P.CONSECUTIVO, EA.CODIGO_ETAPA ) B ON A.PERSONA = B.PERSONA AND A.ETAPA = B.ETAPA \
ORDER BY A.PERSONA, A.ETAPA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_etapa_persona = pd.DataFrame(res , columns=['PERSONA', 'ETAPA','HORAS_ETAPA'])
df_horas_etapa_persona['ETAPA'] = df_horas_etapa_persona['ETAPA'] .fillna('SIN_ETAPA')
pv_horas_etapa_persona = pd.pivot_table(df_horas_etapa_persona, values='HORAS_ETAPA', index=['PERSONA'], columns='ETAPA', fill_value=0)
pv_horas_etapa_persona.head(2)

Duración de Consulta (seg): 0:02:15.480397


ETAPA,ACTCOM,ADMB,ADMBD,ADMCAP,ADMON,ADMSEG,ADMSER,ADMSO,ALEMAN,ANAID,...,TRIBUT,TRINS,TRINSM,TRINT,TRMIG,TRREV,VAC,VAR,VENPRO,VISCLI
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0.0,0,0.0,0,0.0,0,0,0,...,0,0.0,0,0,0.0,0.0,0.0,0,0.0,0.0
8,0,0,0.0,0,0.0,0,0.0,0,0,0,...,0,0.0,0,0,0.0,0.0,120.0,0,0.0,0.0


In [14]:
#Horas en area por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT B.PERSONA, B.AREA, CASE WHEN C.HORAS_AREA IS NULL THEN 0 ELSE C.HORAS_AREA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, A.CONSECUTIVO  AS AREA \
	FROM AREAS  A \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
    AND   P.FECHA_RETIRO IS NULL \
    AND   P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
    AND   P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
	GROUP BY P.CONSECUTIVO,A.CONSECUTIVO  ) B \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.CON_AREA  AS AREA, SUM(EA.DURACION_HORAS) AS HORAS_AREA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN PERSONAS P              ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO  = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
	AND   EA.HORA_FINALIZACION < CAST(CURRENT_TIMESTAMP AS DATE) \
	GROUP BY P.CONSECUTIVO, S.CON_AREA ) C ON B.PERSONA = C.PERSONA AND C.PERSONA =  B.AREA \
ORDER BY B.PERSONA, B.AREA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_area_persona = pd.DataFrame(res , columns=['PERSONA', 'AREA','HORAS_AREA'])
pv_horas_area_persona = pd.pivot_table(df_horas_area_persona, values='HORAS_AREA', index=['PERSONA'], columns='AREA', fill_value=0)
pv_horas_area_persona.head(2)

Duración de Consulta (seg): 0:00:20.639766


AREA,1,2,3,4,5,6,7,8,9,24,...,53,54,55,56,58,59,60,61,62,63
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,97.333333,0,0.0000,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0.000000,0,130.7075,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#Cantidad de pendientes en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_PENDIENTES IS NULL THEN 0 ELSE B.CANTIDAD_PENDIENTES END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND   C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND   P.FECHA_RETIRO IS NULL \
    AND   P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
    AND   P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(PE.CODIGO_PENDIENTE) AS CANTIDAD_PENDIENTES \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A  ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P  ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN PENDIENTES   PE ON A.COD_SERVICIO = PE.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S  ON PE.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
	AND   AH.FECHA_FIN < CAST(CURRENT_TIMESTAMP AS DATE) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   PE.FECHA_DETECCION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) \
	AND   PE.FECHA_DETECCION < CAST(CURRENT_TIMESTAMP AS DATE) \
	GROUP BY P.CONSECUTIVO, S.TIPO) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_pend_tiposerv_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_PENDIENTES'])
pv_pend_tiposerv_persona = pd.pivot_table(df_pend_tiposerv_persona, values='CANTIDAD_PENDIENTES', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_pend_tiposerv_persona.columns = [str(col) + '_PN' for col in pv_pend_tiposerv_persona.columns]
pv_pend_tiposerv_persona.head(2)

Duración de Consulta (seg): 0:00:15.227542


,AC_PN,ADM_PN,AI_PN,AP_PN,CO_PN,CU_PN,GC_PN,HD_PN,IN_PN,LI_PN,NE_PN,OP_PN,PD_PN,PM_PN,PR_PN,RE_PN,SE_PN,SM_PN,SO_PN,TD_PN
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,1628,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,891,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
#Cantidad de riesgos en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_RIESGOS IS NULL THEN 0 ELSE B.CANTIDAD_RIESGOS END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND   C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND   P.FECHA_RETIRO IS NULL \
    AND   P.FECHA_INGRESO <= ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-3) \
    AND   P.GERENCIA_RESPONSABLE  <> 'INTELIGENCIA ARTIFICIAL'  \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(RS.CONSECUTIVO) AS CANTIDAD_RIESGOS \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A      ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P      ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN RIESGOS_SERVICIO RS ON A.COD_SERVICIO = RS.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S      ON RS.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN BETWEEN ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12) AND CAST(CURRENT_TIMESTAMP AS DATE) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   RS.FECHA_REGISTRO BETWEEN ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -12)  AND CAST(CURRENT_TIMESTAMP AS DATE)\
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA, A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_ries_tiposerv_persona = pd.DataFrame(res , columns=['PERSONA', 'TIPO_SERVICIO','CANTIDAD_RIESGOS'])
pv_ries_tiposerv_persona = pd.pivot_table(df_ries_tiposerv_persona, values='CANTIDAD_RIESGOS', index=['PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_ries_tiposerv_persona.columns = [str(col) + '_RS' for col in pv_ries_tiposerv_persona.columns]
pv_ries_tiposerv_persona.head(2)

Duración de Consulta (seg): 0:00:08.881846


,AC_RS,ADM_RS,AI_RS,AP_RS,CO_RS,CU_RS,GC_RS,HD_RS,IN_RS,LI_RS,NE_RS,OP_RS,PD_RS,PM_RS,PR_RS,RE_RS,SE_RS,SM_RS,SO_RS,TD_RS
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
#Edad personas desde su fecha de nacimiento hasta hoy
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, FECHA_NACIMIENTO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NULL AND FECHA_NACIMIENTO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_edad_persona = pd.DataFrame(res , columns=['PERSONA', 'FECHA_NACIMIENTO'])

#Function to calculate people's age
def age(birth_date):
    current_date = date.today() #Y-M-D
    age = current_date.year - birth_date.year - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    return age

df_edad_persona['EDAD'] = df_edad_persona['FECHA_NACIMIENTO'].apply(age)
df_edad_persona.drop(columns = ['FECHA_NACIMIENTO'], inplace = True)
df_edad_persona.head(2)

Duración de Consulta (seg): 0:00:00.555317


,PERSONA,EDAD
0,559,52
1,565,51


In [18]:
#Género de la persona: 0 para M y 1 para F
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, GENERO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NULL AND FECHA_NACIMIENTO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_genero_persona = pd.DataFrame(res , columns=['PERSONA', 'GENERO'])
df_genero_persona.replace(to_replace = ['M', 'F'], value = [0, 1], inplace = True)
df_genero_persona.head(2)

Duración de Consulta (seg): 0:00:00.595162


,PERSONA,GENERO
0,559,0
1,565,1


In [19]:
#Tiempo que la persona ha estado en la compañía
time_start = datetime.datetime.now()

cur.execute("SELECT CONSECUTIVO AS PERSONA, FECHA_INGRESO FROM PERSONAS\
            WHERE FECHA_RETIRO IS NULL AND FECHA_INGRESO IS NOT NULL")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiempo_emp_persona = pd.DataFrame(res , columns=['PERSONA', 'FECHA_INGRESO'])

#Function to calculate people time in the company
def time(date):
    enter_date = date
    current_date = date.today() #Y-M-D
    months = (current_date.year - enter_date.year)*12 + (current_date.month - enter_date.month)
    return months

df_tiempo_emp_persona['TIEMPO_EMP'] = df_tiempo_emp_persona['FECHA_INGRESO'].apply(time)
df_tiempo_emp_persona.drop(columns = ['FECHA_INGRESO'], inplace = True)
df_tiempo_emp_persona.head(2)

Duración de Consulta (seg): 0:00:00.563695


,PERSONA,TIEMPO_EMP
0,559,119
1,565,119


In [20]:
time_start = datetime.datetime.now()

print('asignaciones facturadas x persona:',df_facturas_persona.shape)
print('asignaciones no facturadas x persona:',df_nofacturas_persona.shape)
print('numero tipos servicio x persona:',pv_tiposervicio_persona.shape)
print('horas tipo servicio x persona:',pv_horas_tiposervicio_persona.shape)
print('tecnologias usadas x persona:',pv_tecno_asig_persona.shape)
print('horas en etapa x persona:',pv_horas_etapa_persona.shape)
print('horas en area x persona:',pv_horas_area_persona.shape)
print('pendientes en tipos de servicio x persona:',pv_pend_tiposerv_persona.shape)
print('riesgos en tipos de servicio x persona:',pv_ries_tiposerv_persona.shape)
print('edad en años x persona al momento de retirarse:',df_edad_persona.shape)
print('genero x persona:',df_genero_persona.shape)
print('tiempo en meses en la compañía x persona:',df_tiempo_emp_persona.shape)

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

asignaciones facturadas x persona: (504, 2)
asignaciones no facturadas x persona: (504, 2)
numero tipos servicio x persona: (502, 20)
horas tipo servicio x persona: (502, 20)
tecnologias usadas x persona: (502, 185)
horas en etapa x persona: (502, 374)
horas en area x persona: (502, 44)
pendientes en tipos de servicio x persona: (502, 20)
riesgos en tipos de servicio x persona: (502, 20)
edad en años x persona al momento de retirarse: (606, 2)
genero x persona: (606, 2)
tiempo en meses en la compañía x persona: (606, 2)
Duración de Consulta (seg): 0:00:00.000496


In [21]:
time_start = datetime.datetime.now()

temp_dataset_1 = pd.merge(pv_tiposervicio_persona,
                          pd.merge(df_facturas_persona,df_nofacturas_persona, left_on='PERSONA', right_on='PERSONA'),left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_horas_tiposervicio_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_tecno_asig_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_horas_etapa_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_horas_area_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,pv_pend_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,pv_ries_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(temp_dataset_1,df_edad_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(temp_dataset_2,df_genero_persona, left_on='PERSONA', right_on='PERSONA')
df_dataset_vigentes = pd.merge(temp_dataset_1,df_tiempo_emp_persona, left_on='PERSONA', right_on='PERSONA')
vc_dataset_vigentes = df_dataset_vigentes.values.tolist()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

print(len(df_dataset_vigentes))
df_dataset_vigentes.head()

Duración de Consulta (seg): 0:00:00.140146
502


,PERSONA,AC_TP,ADM_TP,AI_TP,AP_TP,CO_TP,CU_TP,GC_TP,HD_TP,IN_TP,...,PM_RS,PR_RS,RE_RS,SE_RS,SM_RS,SO_RS,TD_RS,EDAD,GENERO,TIEMPO_EMP
0,6,0,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,60,0,367
1,8,1,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,42,1,185
2,21,3,25,1,11,0,0,0,0,0,...,0,0,0,0,245,0,0,41,0,195
3,31,0,18,0,4,0,0,0,0,0,...,0,0,0,0,169,0,0,46,0,150
4,35,0,15,0,0,0,0,0,0,2,...,0,0,0,0,26,9,0,58,0,367


In [22]:
#Exporting the dataset to csv
df_dataset_vigentes.to_csv('data_to_predict_' + str(datetime.datetime.now()) + '.csv', index = False)